# Онлайн-рекомендации

In [1]:
import pandas as pd
events = pd.read_parquet("events.par")

In [2]:
train_test_global_time_split_date = pd.to_datetime("2017-08-01")

events['started_at'] = pd.to_datetime(events.started_at)
events['read_at'] = pd.to_datetime(events.read_at)

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"].dt.date < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()

events_inference = pd.concat([events_train, events_labels])

In [3]:
import numpy as np
import pandas as pd
items = pd.read_parquet("items.par")
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [4]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] =  user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] =        item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"]) # ваш код здесь #
events_test["item_id_enc"] =  item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_1404/542547798.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_1404/542547798.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] =  user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_1404/542547798.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [5]:
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [6]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(
    factors=50, iterations=50,
    regularization=0.05, random_state=0
)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

## Шаг 1. Набор похожих объектов

Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки implicit, у которого на такой случай есть удобный метод similar_items (подробнее о нём вы можете прочитать в официальной документации).

Воспользуемся им и получим по 10 самых похожих айтемов.

## Задание 1 из 6

Дополните код ниже, чтобы получить набор похожих объектов в similar_items. Вы можете подглядеть решение в уроке «Коллаборативная фильтрация: ALS» — там вы реализовывали похожую логику для получения персональных рекомендаций.

In [7]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

In [8]:
similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()
}) # ваш код здесь #)

In [9]:
similar_items.head()

,item_id_enc,sim_item_id_enc,score
0,2460,"[2460, 2458, 806, 2459, 12528, 1147, 7852, 618...","[0.9999999403953552, 0.9224898815155029, 0.874..."
1,38691,"[38691, 39575, 40111, 25112, 32177, 34430, 367...","[1.0000001192092896, 0.9343445897102356, 0.930..."
2,38867,"[38867, 38023, 38951, 5992, 3865, 10539, 28584...","[1.0, 0.9388757348060608, 0.9345316886901855, ..."
3,39109,"[39109, 37674, 39384, 40645, 17054, 36002, 394...","[0.9999998211860657, 0.9593728184700012, 0.947..."
4,35638,"[35638, 37837, 41337, 39997, 31205, 25389, 324...","[1.0000001192092896, 0.9470844268798828, 0.944..."


In [10]:
similar_items = similar_items.explode(["sim_item_id_enc", "score"]) # ваш код здесь #
similar_items

,item_id_enc,sim_item_id_enc,score
0,2460,2460,1.0
0,2460,2458,0.92249
0,2460,806,0.874765
0,2460,2459,0.873763
0,2460,12528,0.850654
...,...,...,...
41473,38365,37490,0.53492
41473,38365,23306,0.515321
41473,38365,35631,0.50771
41473,38365,23687,0.496325


In [11]:
similar_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456214 entries, 0 to 41473
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   item_id_enc      456214 non-null  int64 
 1   sim_item_id_enc  456214 non-null  object
 2   score            456214 non-null  object
dtypes: int64(1), object(2)
memory usage: 13.9+ MB


In [12]:
# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype("int") # ваш код здесь #
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"]) # ваш код здесь #
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"]) # ваш код здесь #
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [62]:
tmp

,score,item_id_1,item_id_2
0,0.922490,22034,22026
0,0.874765,22034,6882
0,0.873763,22034,22028
0,0.850654,22034,364089
0,0.835730,22034,9827
...,...,...,...
41473,0.534920,21847032,19904043
41473,0.515321,21847032,6167746
41473,0.507710,21847032,17908487
41473,0.496325,21847032,6349976


In [63]:
tmp = similar_items.copy()
tmp.set_index('item_id_1')
tmp.loc[7126] #.head(5)

,score,item_id_1,item_id_2
7126,0.758994,25624,6727758
7126,0.757831,25624,2230670
7126,0.753783,25624,13563459
7126,0.752194,25624,600791
7126,0.752172,25624,9864913
7126,0.748368,25624,6092826
7126,0.741506,25624,18514068
7126,0.739627,25624,23433546
7126,0.738119,25624,2757012
7126,0.736407,25624,14082


In [59]:
# Укажите идентификатор объекта, наиболее похожего на объект 7126.
similar_items.query('item_id_1 == 7126')

,score,item_id_1,item_id_2
2352,0.948725,7126,7190
2352,0.940997,7126,24280
2352,0.930144,7126,1953
2352,0.925066,7126,58696
2352,0.916340,7126,38296
2352,0.916015,7126,2932
2352,0.913951,7126,7184
2352,0.911433,7126,387749
2352,0.909872,7126,7733
2352,0.909454,7126,30597


In [14]:
similar_items

,score,item_id_1,item_id_2
0,0.922490,22034,22026
0,0.874765,22034,6882
0,0.873763,22034,22028
0,0.850654,22034,364089
0,0.835730,22034,9827
...,...,...,...
41473,0.534920,21847032,19904043
41473,0.515321,21847032,6167746
41473,0.507710,21847032,17908487
41473,0.496325,21847032,6349976


Полученный набор вскоре вам снова понадобится, так что сохраните его в файле: 

In [16]:
similar_items.to_parquet("similar_items.parquet") 

Полезно убедиться, что полученный набор действительно содержит похожие данные. Например, можно оценить глазами списки похожих объектов для каких-то уже известных. Создадим для этой цели функцию print_sim_items.

In [56]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(
        items[item_columns_to_use].set_index("item_id"),
        left_on="item_id_2",
        right_index=True
    )
    display(si)

Попробуйте оценить похожие айтемы для следующих известных книг (числа — идентификаторы item_id):
- 7144: Ф. М. Достоевский «Преступление и наказание»;
- 16299: Агата Кристи «Десять негритят»;
- 3: Джоан Роулинг «Гарри Поттер и философский камень»;
- 18135: Уильям Шекспир «Ромео и Джульетта»;
- 17245: Брэм Стокер «Дракула».

Для этого вызовите функцию, указанную выше, например:

In [57]:
print_sim_items(7144, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1909078,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,"{'Classics': 15812, 'Fiction': 8028, 'Cultural...",4.19,390293


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
6069,0.964479,7144,12505,"Fyodor Dostoyevsky, Anna Brailovsky, Constance...",The Idiot,"{'Classics': 4036, 'Fiction': 2576}",4.18,76392
6069,0.953918,7144,12857,"Fyodor Dostoyevsky, Constance Garnett",The Gambler,"{'Classics': 946, 'Fiction': 729, 'Cultural-Ru...",3.88,22024
6069,0.952009,7144,67326,Fyodor Dostoyevsky,Poor Folk,"{'Classics': 320, 'Fiction': 235, 'Literature-...",3.73,4957
6069,0.946847,7144,5508624,Leo Tolstoy,Family Happiness,"{'Classics': 140, 'Fiction': 112, 'Cultural-Ru...",3.85,3337
6069,0.939762,7144,4934,"Fyodor Dostoyevsky, Fyodor Dostoyevsky, Richar...",The Brothers Karamazov,"{'Classics': 7496, 'Fiction': 5491, 'Cultural-...",4.31,158410
6069,0.938018,7144,17877,"Fyodor Dostoyevsky, Constance Garnett",The House of the Dead,"{'Classics': 533, 'Fiction': 441, 'Cultural-Ru...",4.04,8548
6069,0.937007,7144,929782,"Jack London, Andrew Sinclair",Martin Eden,"{'Classics': 435, 'Fiction': 405, 'Literature-...",4.39,13257
6069,0.936360,7144,28382,Nikolai Gogol,Diary of a Madman and Other Stories,"{'Classics': 284, 'Fiction': 243, 'Short Stori...",4.09,6241
6069,0.936320,7144,17690,"Franz Kafka, Max Brod, Willa Muir, Edwin Muir",The Trial,"{'Classics': 4607, 'Fiction': 4173, 'Literatur...",3.98,135862
6069,0.934541,7144,63038,Victor Hugo,The Man Who Laughs,"{'Classics': 352, 'Fiction': 176, 'Cultural-Fr...",4.22,5449


## Шаг 2. Сервис Feature Store

Теперь сделаем так, чтобы набор стал доступен сервису рекомендаций.
Для этого создадим новый сервис, который при запуске будет загружать набор похожих объектов из файла "similar_items.parquet" и отдавать список похожих объектов через метод `/similar_items`.

## Задание 2 из 6

Дополните код ниже, чтобы получить работоспособный сервис, возвращающий список похожих объектов через метод /similar_items.
Сохраните код сервиса в файле `features_service.py`.

In [27]:
!cat service/features_service.py

import logging
from contextlib import asynccontextmanager

import pandas as pd
from fastapi import FastAPI

logger = logging.getLogger("uvicorn.error")

class SimilarItems:

    def __init__(self):

        self._similar_items = None

    def load(self, path, **kwargs):
        """
        Загружаем данные из файла
        """

        logger.info(f"Loading data")
        self._similar_items = pd.read_parquet(path, **kwargs)
        self._similar_items = self._similar_items[kwargs['columns']]
        logger.info(f"Loaded")

    def get(self, item_id: int, k: int = 10):
        """
        Возвращает список похожих объектов
        """
        try:
            i2i = self._similar_items.loc[item_id].head(k)
            i2i = i2i[["item_id_2", "score"]].to_dict(orient="list")
        except KeyError:
            logger.error("No recommendations found")
            i2i = {"item_id_2": [], "score": {}}

        return i2i

sim_items_store = SimilarItems()

@asynccontextmanager
async def lif

In [66]:
import requests

features_store_url = "http://127.0.0.1:8010"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"item_id": 17245}

resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
if resp.status_code == 200:
    similar_items = resp.json()
else:
    similar_items = None
    print(f"status code: {resp.status_code}")
    
print(similar_items) 

{'item_id_2': [480204, 51496, 93261, 295, 2623, 18254, 7190, 24213, 2932, 1953], 'score': [0.9288230538368225, 0.9003370404243469, 0.8989384174346924, 0.8977058529853821, 0.8964701294898987, 0.8959931135177612, 0.8868994116783142, 0.8819113373756409, 0.8783923387527466, 0.870232105255127]}


## Шаг 3. Сервис Event Store

Чтобы выполнить второй пункт алгоритма («для онлайн-взаимодействия пользователя с каким-то объектом можно использовать список похожих на него объектов»), необходим компонент, умеющий сохранять и выдавать последние события пользователя, — это Event Store. 

Реализуем его также в виде сервиса. В данном случае под взаимодействием пользователя с объектом будем подразумевать любое положительное событие, например: просмотр страницы с книгой, лайк, добавление в избранное и т.п.

## Задание 3 из 6

Дополните код сервиса так, чтобы он по методу `/put` сохранял пару значений `user_id` и `item_id` как событие, а по методу `/get` возвращал события (первыми — самые последние).

Сохраните код сервиса в файле `events_service.py`.

In [30]:
!cat service/events_service.py

from fastapi import FastAPI

class EventStore:

    def __init__(self, max_events_per_user=10):

        self.events = {}
        self.max_events_per_user = max_events_per_user

    def put(self, user_id, item_id):
        """
        Сохраняет событие
        """

        user_events = self.events.get(user_id, [])  # ваш код здесь #
        self.events[user_id] = [item_id] + user_events[: self.max_events_per_user]

    def get(self, user_id, k):
        """
        Возвращает события для пользователя
        """
        user_events = self.events.get(user_id, [])[:k]  # ваш код здесь #

        return user_events

events_store = EventStore() # ваш код здесь #

# создаём приложение FastAPI
app = FastAPI(title="events")

@app.post("/put")
async def put(user_id: int, item_id: int):
    """
    Сохраняет событие для user_id, item_id
    """

    events_store.put(user_id, item_id)

    return {"result": "ok"}

@app.post("/get")
async def get(user_id: int, k: int = 10):
    """
    Возвращае

Проверьте, что для пользователя `1127794` в Event Store нет никаких событий.
Затем сохраните для этого пользователя последовательно четыре события с объектами:
18734992, 18734992, 7785, 4731479. 
После чего получите для того же пользователя последние три события:

In [31]:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1127794}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result) 

{'events': []}


In [32]:
import requests

events_store_url = "http://127.0.0.1:8020"

for id_ in [18734992, 18734992, 7785, 4731479]:
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    params = {"user_id": 1127794, "item_id": id_}
    
    resp = requests.post(events_store_url + "/put", headers=headers, params=params)
    if resp.status_code == 200:
        result = resp.json()
    else:
        result = None
        print(f"status code: {resp.status_code}")
        
    print(result) 

{'result': 'ok'}
{'result': 'ok'}
{'result': 'ok'}
{'result': 'ok'}


In [34]:
resp = requests.post(events_store_url + "/get", 
                     headers=headers, 
                     params={"user_id": 1127794, "k": 3})
print(resp.json()) 

{'events': [4731479, 7785, 18734992]}


## Шаг 4. Доработка сервиса рекомендаций

Доработаем уже имеющийся код сервиса из предыдущего урока так, чтобы он позволял получить онлайн-рекомендации для последнего события пользователя. Для этого добавьте в код сервиса рекомендаций новые переменные, указывающие на адреса новых сервисов:

- `features_store_url = "http://127.0.0.1:8010"`
- `events_store_url = "http://127.0.0.1:8020"`
  
Затем реализуйте новый метод /recommendations_online, который будет выдавать список похожих объектов для последнего события пользователя (если оно есть).

## Задание 4 из 6

Дополните код нового метода так, чтобы для последнего события пользователя, если оно есть, возвращался список похожих объектов. Это и будут онлайн-рекомендации.

In [44]:
!cat service/recommendation_service.py

import logging
import requests
from fastapi import FastAPI
from contextlib import asynccontextmanager
from handler import Recommendations

logger = logging.getLogger("uvicorn.error")
rec_store = Recommendations()

features_store_url = "http://0.0.0.0:8010"
events_store_url = "http://0.0.0.0:8020"

@asynccontextmanager
async def lifespan(app: FastAPI):
    # код ниже (до yield) выполнится только один раз при запуске сервиса
    logger.info("Starting")
    rec_store.load(
        "personal",
        "datasets/final_recommendations.parquet", # ваш код здесь #
        columns=["user_id", "item_id", "rank"],
    )
    rec_store.load(
        "default",
        "datasets/top_recs.parquet", # ваш код здесь #,
        columns=["item_id", "rank"],
    )
    yield
    info = rec_store.stats()
    logger.info(info)
    # этот код выполнится только один раз при остановке сервиса
    logger.info("Stopping")
    
# создаём приложение FastAPI
app = FastAPI(title="recommendations", lifespan=lifespan)


Протестируйте работу нового метода. Вы получите онлайн-рекомендации (длиной 3) для пользователя 1291248:

In [39]:
recommendations_url = 'http://158.160.88.42:8000'
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)

{'recs': []}


Список пустой. Это ожидаемо, так как для пользователя в Event Store пока нет никаких событий, чтобы по ним получить онлайн-рекомендации.

Добавим событие:

In [40]:
params = {"user_id": 1291248, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)

И снова получим онлайн-рекомендации для пользователя 1291248:

In [69]:
# Что возвращает метод /recommendations_online (с параметром k=3) для пользователя 1291248 после добавления для него события с объектом 17245?

recommendations_url = 'http://158.160.88.42:8000'
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)

{'recs': [608474, 8921, 3590]}


**Шаг 5. Добавим разнообразия в онлайн-рекомендации**

Текущая реализация онлайн-рекомендаций позволяет получать их только для последнего события, что очень просто и прямолинейно: если пользователь просмотрит новую книгу, то у него сменятся все онлайн-рекомендации.

Доработаем алгоритм так, чтобы онлайн-рекомендации учитывали три последних объекта, с которыми взаимодействовал пользователь. Например, так:

- Для каждого события из последних трёх получим список похожих объектов.
- Объединим полученные списки по какому-то правилу. Правило выберем простое: все полученные похожие объекты сортируются по убыванию score, а из упорядоченного списка удаляются дубликаты, оставляя только первое вхождение.

## Задание 5 из 6

Дополните новую версию реализации метода /recommendations_online так, чтобы онлайн-рекомендации возвращались для трёх последних событий.

In [45]:
!cat service/recommendation_service.py

import logging
import requests
from fastapi import FastAPI
from contextlib import asynccontextmanager
from handler import Recommendations

logger = logging.getLogger("uvicorn.error")
rec_store = Recommendations()

features_store_url = "http://0.0.0.0:8010"
events_store_url = "http://0.0.0.0:8020"

def dedup_ids(combined: list) -> list:
    unique = []
    for item in combined:
        if item not in unique:
            unique.append(item)
    return unique

@asynccontextmanager
async def lifespan(app: FastAPI):
    # код ниже (до yield) выполнится только один раз при запуске сервиса
    logger.info("Starting")
    rec_store.load(
        "personal",
        "datasets/final_recommendations.parquet", # ваш код здесь #
        columns=["user_id", "item_id", "rank"],
    )
    rec_store.load(
        "default",
        "datasets/top_recs.parquet", # ваш код здесь #,
        columns=["item_id", "rank"],
    )
    yield
    info = rec_store.stats()
    logger.info(info)
    # этот код выполн

Протестируем метод для пользователя 1291248, сгенерировав для него несколько событий и получив пять онлайн-рекомендаций.

In [70]:
user_id = 1291248
event_item_ids = [41899, 102868, 5472, 5907]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})
                         
params = {"user_id": user_id, 'k': 5}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)

{'recs': [608474, 8921, 3590, 194373, 736131]}


Учитывая утверждение, что офлайн- и онлайн-рекомендации не исключают, а дополняют друг друга, доработаем сервис рекомендаций так, чтобы результат его работы мог включать в себя оба типа.

Предлагаем такую простую схему:
- онлайн-рекомендации занимают нечётные места,
- офлайн-рекомендации занимают чётные места.

Подобные схемы расстановок разнотипных элементов по различным местам ещё называют «смешиванием» (англ. blending).

Начнём с того, что переименуем метод /recommendations и его функцию в recommendations_offline/. Код функции тот же.

```@app.post("/recommendations_offline")
async def recommendations_offline(user_id: int, k: int = 100):
    """
    Возвращает список офлайн-рекомендаций длиной k для пользователя user_id
    """
    
        ...
```

И реализуем новый метод /recommendations — уже как объединяющий оба типа рекомендаций.

## Задание 6 из 6

Доработайте код обновлённого метода /recommendations так, чтобы реализовать предложенную выше схему блендинга.

In [52]:
!cat service/recommendation_service.py

import logging
import requests
from fastapi import FastAPI
from contextlib import asynccontextmanager
from handler import Recommendations

logger = logging.getLogger("uvicorn.error")
rec_store = Recommendations()

features_store_url = "http://0.0.0.0:8010"
events_store_url = "http://0.0.0.0:8020"

def dedup_ids(combined: list) -> list:
    unique = []
    for item in combined:
        if item not in unique:
            unique.append(item)
    return unique

@asynccontextmanager
async def lifespan(app: FastAPI):
    # код ниже (до yield) выполнится только один раз при запуске сервиса
    logger.info("Starting")
    rec_store.load(
        "personal",
        "datasets/final_recommendations.parquet", # ваш код здесь #
        columns=["user_id", "item_id", "rank"],
    )
    rec_store.load(
        "default",
        "datasets/top_recs.parquet", # ваш код здесь #,
        columns=["item_id", "rank"],
    )
    yield
    info = rec_store.stats()
    logger.info(info)
    # этот код выполн

На примере пользователя 1291250 протестируем доработанный сервис.
Сначала сгенерируем онлайн-события:

In [53]:
user_id = 1291250
event_item_ids =  [7144, 16299, 5907, 18135]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})

Получим 10 рекомендаций каждого типа для данного пользователя:

In [71]:
params = {"user_id": 1291250, 'k': 10}
resp_offline = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
resp_online = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
resp_blended = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

recs_offline = resp_offline.json()["recs"]
recs_online = resp_online.json()["recs"]
recs_blended = resp_blended.json()["recs"]

print(recs_offline)
print(recs_online)
print(recs_blended)

[22557272, 29056083, 18007564, 18143977, 16096824, 3, 9460487, 38447, 15881, 11235712]
[34, 15241, 8852, 18512, 1420, 7728, 17250, 1622, 12296, 13006]
[22557272, 15241, 18007564, 18512, 16096824, 7728, 9460487, 1622, 15881, 13006]


Качество рекомендаций также можно оценить выборочно: посмотрев, что рекомендации книг в целом адекватны, по авторам, названиям. Пример кода для этого приведён ниже.

In [72]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)
    
print("Онлайн-события")
display_items(event_item_ids)
print("Офлайн-рекомендации")
display_items(recs_offline)
print("Онлайн-рекомендации")
display_items(recs_online)
print("Рекомендации")
display_items(recs_blended)

Онлайн-события


,item_id,author,title,genre_and_votes,average_rating,ratings_count
0,41899,"Newt Scamander, J.K. Rowling",Fantastic Beasts and Where to Find Them,"{'Fantasy': 8527, 'Fiction': 1924, 'Young Adul...",3.96,194645
1,102868,Arthur Conan Doyle,A Study in Scarlet,"{'Classics': 6361, 'Mystery': 5360, 'Fiction':...",4.15,207773
2,5472,"George Orwell, Christopher Hitchens",Animal Farm / 1984,"{'Classics': 1149, 'Fiction': 828, 'Science Fi...",4.26,120269
3,5907,J.R.R. Tolkien,The Hobbit,"{'Fantasy': 50050, 'Classics': 16860, 'Fiction...",4.25,2099680


Офлайн-рекомендации


,item_id,author,title,genre_and_votes,average_rating,ratings_count
0,22557272,Paula Hawkins,The Girl on the Train,"{'Fiction': 9793, 'Mystery': 9190, 'Thriller':...",3.88,1076144
1,29056083,"John Tiffany, Jack Thorne, J.K. Rowling",Harry Potter and the Cursed Child - Parts One ...,"{'Fantasy': 14466, 'Fiction': 4232, 'Young Adu...",3.74,288018
2,18007564,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",4.39,435440
3,18143977,Anthony Doerr,All the Light We Cannot See,"{'Historical-Historical Fiction': 13679, 'Fict...",4.31,498685
4,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",4.28,182581
5,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad...",4.45,4765497
6,9460487,Ransom Riggs,Miss Peregrine’s Home for Peculiar Children (M...,"{'Fantasy': 12454, 'Young Adult': 9293, 'Ficti...",3.89,641884
7,38447,Margaret Atwood,The Handmaid's Tale,"{'Fiction': 15424, 'Classics': 9937, 'Science ...",4.07,648783
8,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict...",4.38,1821802
9,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)","{'Young Adult': 10539, 'Fantasy': 9237, 'Scien...",4.15,441530


Онлайн-рекомендации


,item_id,author,title,genre_and_votes,average_rating,ratings_count
0,34,J.R.R. Tolkien,The Fellowship of the Ring (The Lord of the Ri...,"{'Fantasy': 38907, 'Classics': 10145, 'Fiction...",4.34,1813229
1,15241,"J.R.R. Tolkien, Peter S. Beagle","The Two Towers (The Lord of the Rings, #2)","{'Fantasy': 28091, 'Fiction': 6763, 'Classics'...",4.42,490005
2,8852,William Shakespeare,Macbeth,"{'Classics': 16116, 'Plays': 8310, 'Fiction': ...",3.88,502298
3,18512,J.R.R. Tolkien,"The Return of the King (The Lord of the Rings,...","{'Fantasy': 26865, 'Fiction': 6501, 'Classics'...",4.51,473101
4,1420,"William Shakespeare, Harold Bloom, Rex Gibson",Hamlet,"{'Classics': 17549, 'Plays': 8817, 'Fiction': ...",4.01,526122
5,7728,"Sophocles, J.E. Thomas","Antigone (The Theban Plays, #3)","{'Classics': 3847, 'Plays': 2667, 'Drama': 897...",3.61,69075
6,17250,"Arthur Miller, Christopher Bigsby",The Crucible,"{'Classics': 7902, 'Plays': 4768, 'Fiction': 2...",3.55,247565
7,1622,"William Shakespeare, Paul Werstine, Barbara A....",A Midsummer Night's Dream,"{'Classics': 12032, 'Plays': 6438, 'Fiction': ...",3.94,340695
8,12296,"Nathaniel Hawthorne, Thomas E. Connolly, Faust...",The Scarlet Letter,"{'Classics': 19456, 'Fiction': 6716}",3.37,515452
9,13006,"William Shakespeare, Roma Gill",Julius Caesar,"{'Classics': 5864, 'Plays': 3637, 'Fiction': 1...",3.66,121890


Рекомендации


,item_id,author,title,genre_and_votes,average_rating,ratings_count
0,22557272,Paula Hawkins,The Girl on the Train,"{'Fiction': 9793, 'Mystery': 9190, 'Thriller':...",3.88,1076144
1,15241,"J.R.R. Tolkien, Peter S. Beagle","The Two Towers (The Lord of the Rings, #2)","{'Fantasy': 28091, 'Fiction': 6763, 'Classics'...",4.42,490005
2,18007564,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",4.39,435440
3,18512,J.R.R. Tolkien,"The Return of the King (The Lord of the Rings,...","{'Fantasy': 26865, 'Fiction': 6501, 'Classics'...",4.51,473101
4,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",4.28,182581
5,7728,"Sophocles, J.E. Thomas","Antigone (The Theban Plays, #3)","{'Classics': 3847, 'Plays': 2667, 'Drama': 897...",3.61,69075
6,9460487,Ransom Riggs,Miss Peregrine’s Home for Peculiar Children (M...,"{'Fantasy': 12454, 'Young Adult': 9293, 'Ficti...",3.89,641884
7,1622,"William Shakespeare, Paul Werstine, Barbara A....",A Midsummer Night's Dream,"{'Classics': 12032, 'Plays': 6438, 'Fiction': ...",3.94,340695
8,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict...",4.38,1821802
9,13006,"William Shakespeare, Roma Gill",Julius Caesar,"{'Classics': 5864, 'Plays': 3637, 'Fiction': 1...",3.66,121890
